In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

N_RECOMMENDATIONS = 10


# ==========================================
# STEP 1: ANALYZE TEST USERS FIRST
# ==========================================
print("\n[STEP 1] Loading and analyzing test users...")

test_df = pd.read_csv('/kaggle/input/yassir-ai-market-challenge/test_new_version.csv')
print(f"Test users: {len(test_df):,}")

# Load orders to check test users
orders_df = pd.read_csv(
    '/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/orders_df.csv',
    usecols=['order_id', 'user_id']
)

test_user_ids = set(test_df['user_id'].values)

# Check: Do test users have order history?
test_users_with_history = orders_df[orders_df['user_id'].isin(test_user_ids)]['user_id'].nunique()
print(f"Test users with order history: {test_users_with_history:,}/{len(test_user_ids):,}")

cold_start_ratio = 1 - (test_users_with_history / len(test_user_ids))
print(f"Cold start users: {cold_start_ratio:.1%}")

# Get test users' orders
test_user_order_ids = set(orders_df[orders_df['user_id'].isin(test_user_ids)]['order_id'].values)
print(f"Test users' total orders: {len(test_user_order_ids):,}")

# ==========================================
# STEP 2: LOAD ORDER-PRODUCTS (SMART SAMPLING)
# ==========================================
print(f"\n[STEP 2] Loading order-products...")

# Strategy: Load ALL test user data + sample of other users
print("  Loading in chunks...")

all_chunks = []
chunk_count = 0

for chunk in pd.read_csv(
    '/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/orders_products_df.csv',
    chunksize=5000000,
    usecols=['order_id', 'product_id']
):
    chunk_count += 1
    print(f"  Chunk {chunk_count}: {len(chunk):,} rows")
    
    # Keep ALL test user orders
    test_user_chunk = chunk[chunk['order_id'].isin(test_user_order_ids)]
    
    # Sample 20% of other orders (for product stats)
    other_chunk = chunk[~chunk['order_id'].isin(test_user_order_ids)]
    other_sample = other_chunk.sample(frac=0.2, random_state=42) if len(other_chunk) > 0 else other_chunk
    
    combined = pd.concat([test_user_chunk, other_sample], ignore_index=True)
    all_chunks.append(combined)
    
    print(f"    Test: {len(test_user_chunk):,}, Sample: {len(other_sample):,}")
    
    # Memory management - only process 6 chunks max
    if chunk_count >= 7:
        print("  Stopping at 7 chunks for memory...")
        break

op_df = pd.concat(all_chunks, ignore_index=True)
del all_chunks
print(f"\n✓ Total order-products: {len(op_df):,}")

# Add user_id
print("  Adding user_id...")
order_to_user = dict(zip(orders_df['order_id'], orders_df['user_id']))
op_df['user_id'] = op_df['order_id'].map(order_to_user)
op_df = op_df.dropna(subset=['user_id'])

print(f"✓ With user_id: {len(op_df):,}")

# ==========================================
# STEP 3: SIMPLE FEATURES
# ==========================================
print(f"\n[STEP 3] Computing simple features...")

# User-product frequency
print("  User-product frequency...")
user_product_freq = op_df.groupby(['user_id', 'product_id']).size()

# Product popularity (from all data)
print("  Product popularity...")
product_popularity = op_df['product_id'].value_counts()
top_500 = product_popularity.head(500).index.tolist()

print(f"✓ User-product pairs: {len(user_product_freq):,}")
print(f"✓ Unique products: {len(product_popularity):,}")

# User's products
user_products = defaultdict(list)
for (uid, pid), count in user_product_freq.items():
    user_products[uid].append((pid, count))

# Sort by frequency
for uid in user_products:
    user_products[uid] = sorted(user_products[uid], key=lambda x: x[1], reverse=True)

print(f"✓ Users with history: {len(user_products):,}")

# ==========================================
# STEP 4: CATEGORY FEATURES (SIMPLE)
# ==========================================
print(f"\n[STEP 4] Loading categories...")

category_df = pd.read_csv('/kaggle/input/yassir-ai-market-challenge/yassir_marekt_data_09_2025/yassir_marekt_data_09_2025 2/category_df.csv')

cat_cols = list(category_df.columns)
product_to_category = dict(zip(category_df[cat_cols[0]], category_df[cat_cols[1]]))

print(f"✓ Products with category: {len(product_to_category):,}")

# User's favorite categories
user_favorite_categories = {}
for uid in user_products:
    cat_counter = Counter()
    for pid, count in user_products[uid]:
        if pid in product_to_category:
            cat_counter[product_to_category[pid]] += count
    
    if cat_counter:
        user_favorite_categories[uid] = cat_counter.most_common(3)

print(f"✓ Users with category preferences: {len(user_favorite_categories):,}")

# Category's top products
category_top_products = defaultdict(list)
op_with_cat = op_df.copy()
op_with_cat['category'] = op_with_cat['product_id'].map(product_to_category)
cat_prod_counts = op_with_cat.groupby(['category', 'product_id']).size()

for cat in cat_prod_counts.index.get_level_values(0).unique():
    if pd.notna(cat):
        top = cat_prod_counts[cat].nlargest(30).index.tolist()
        category_top_products[cat] = top

print(f"✓ Categories: {len(category_top_products)}")

del op_with_cat, op_df, orders_df
import gc
gc.collect()

# ==========================================
# STEP 5: SIMPLE RECOMMENDER (EXACT ORIGINAL)
# ==========================================
print(f"\n[STEP 5] Creating simple recommender...")

def simple_recommend(user_id, n=N_RECOMMENDATIONS):
    """
    EXACT ORIGINAL FUNCTION THAT GOT 0.21998:
    1. Return user's most frequent products (reorder)
    2. Add popular products from user's favorite categories
    3. Fill with overall popular products
    """
    recommendations = []
    seen = set()
    
    # STEP 1: User's frequent products (REORDER - most important!)
    if user_id in user_products:
        for pid, count in user_products[user_id][:n]:  # ORIGINAL: variable number up to n
            recommendations.append(pid)
            seen.add(pid)
    
    # STEP 2: Popular products from user's favorite categories
    if user_id in user_favorite_categories and len(recommendations) < n:
        for cat, cat_count in user_favorite_categories[user_id]:
            if cat in category_top_products:
                for prod in category_top_products[cat]:
                    if prod not in seen:
                        recommendations.append(prod)
                        seen.add(prod)
                        if len(recommendations) >= n:
                            break
            if len(recommendations) >= n:
                break
    
    # STEP 3: Fill with popular products
    if len(recommendations) < n:
        for prod in top_500:
            if prod not in seen:
                recommendations.append(prod)
                seen.add(prod)
                if len(recommendations) >= n:
                    break
    
    return recommendations[:n]

print(f"✓ Recommender ready")

# ==========================================
# STEP 6: GENERATE SUBMISSION
# ==========================================
print(f"\n[STEP 6] Generating submission...")

predictions = []
test_users = test_df['user_id'].values

print(f"Processing {len(test_users):,} users...")
for i, uid in enumerate(test_users):
    if i % 5000 == 0 and i > 0:
        print(f"  {i:,}/{len(test_users):,}...")
    
    recs = simple_recommend(uid, N_RECOMMENDATIONS)
    predictions.append({
        'user_id': uid,
        'product_id': ' '.join(map(str, recs))
    })

submission_df = pd.DataFrame(predictions)
submission_file = 'exact_original_021998.csv'
submission_df.to_csv(submission_file, index=False)

print(f"\n✓ Saved: {submission_file}")

# ==========================================
# STEP 7: ANALYSIS
# ==========================================
print(f"\n{'='*60}")
print("EXACT ORIGINAL 0.21998 CODE")
print(f"{'='*60}")

# Check recommendation stats
all_recs = []
for p in predictions[:1000]:
    all_recs.extend(map(int, p['product_id'].split()))

rec_counter = Counter(all_recs)

print(f"\n📊 STATS:")
print(f"  Total predictions: {len(predictions):,}")
print(f"  Unique products recommended: {len(set(all_recs)):,}")

print(f"\n🎯 EXACT ORIGINAL STRATEGY:")
print(f"  1. User's most purchased products (reorder)")
print(f"  2. Popular from user's favorite categories") 
print(f"  3. Overall popular products")

print(f"\nTop 10 recommended products:")
for prod, count in rec_counter.most_common(10):
    pct = count / len(predictions[:1000]) * 100
    print(f"  Product {prod}: {count} times ({pct:.1f}%)")


print(f"\n{'='*60}")
print(f"✅ Upload '{submission_file}'")
print(f"Expected: 0.21998 (original score)")
print(f"{'='*60}")

EXACT ORIGINAL 0.21998 CODE

[STEP 1] Loading and analyzing test users...
Test users: 75,486
Test users with order history: 75,486/75,486
Cold start users: 0.0%
Test users' total orders: 926,770

[STEP 2] Loading order-products...
  Loading in chunks...
  Chunk 1: 5,000,000 rows
    Test: 1,422,608, Sample: 715,478
  Chunk 2: 5,000,000 rows
    Test: 1,424,776, Sample: 715,045
  Chunk 3: 5,000,000 rows
    Test: 1,425,521, Sample: 714,896
  Chunk 4: 5,000,000 rows
    Test: 1,420,128, Sample: 715,974
  Chunk 5: 5,000,000 rows
    Test: 1,429,971, Sample: 714,006
  Chunk 6: 5,000,000 rows
    Test: 1,431,823, Sample: 713,635
  Chunk 7: 1,536,952 rows
    Test: 436,740, Sample: 220,042
  Stopping at 7 chunks for memory...

✓ Total order-products: 13,500,643
  Adding user_id...
✓ With user_id: 13,500,643

[STEP 3] Computing simple features...
  User-product frequency...
  Product popularity...
✓ User-product pairs: 7,175,440
✓ Unique products: 49,074
✓ Users with history: 204,468

[STEP 4